In [8]:
import numpy as np
import itertools
import json
import pandas as pd

from ray.rllib.algorithms.dqn.dqn import DQNConfig
from ray.rllib.algorithms.ppo.ppo import PPOConfig
from ray.rllib.algorithms.a3c.a3c import A3CConfig
from ray.rllib.algorithms.a2c.a2c import A2CConfig
from ray.rllib.algorithms.appo.appo import APPOConfig
from ray.rllib.algorithms.impala.impala import ImpalaConfig
from ray.rllib.algorithms.pg.pg import PGConfig
from ray.rllib.algorithms.sac.sac import SACConfig


from ray.tune.registry import register_env


import tensorflow as tf
from towerofhanoienv import TowerOfHanoiEnv, EmbeddingEnv, FlattenEnv
from preprocessing import AutoEncoder, Dataset


import pickle
import warnings
warnings.filterwarnings("ignore")


In [9]:
checkpoint_root = "tmp/dqn/towerofhanoi"

In [23]:
config = SACConfig()
config = config.rollouts(num_rollout_workers=3) 
config =config.framework("tf2") 
config = config.environment(TowerOfHanoiEnv, env_config={"num_disks": 3}, disable_env_checking=True) 
config = config.training(model={"_disable_preprocessor_api": False,"use_lstm":False}, lr=0.01)

agent = config.build()  
agent.train()  

results = []
episode_data = []
episode_json = []

for n in range(15):
    result = agent.train()
    results.append(result)

    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"],
        "episode_reward_max": result["episode_reward_max"],
        "episode_len_mean": result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)

    print(f'{n:3d}: Min/Mean/Max reward/episode length mean: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}/{result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

(RolloutWorker pid=1264) /Users/sophialawal/opt/anaconda3/envs/tf/lib/python3.10/site-packages/gymnasium/spaces/multi_discrete.py:177: UserWarning: WARN: Getting the length of a multi-dimensional MultiDiscrete space.
(RolloutWorker pid=1264)   gym.logger.warn(
(RolloutWorker pid=1263) /Users/sophialawal/opt/anaconda3/envs/tf/lib/python3.10/site-packages/gymnasium/spaces/multi_discrete.py:177: UserWarning: WARN: Getting the length of a multi-dimensional MultiDiscrete space.
(RolloutWorker pid=1263)   gym.logger.warn(
(RolloutWorker pid=1265) /Users/sophialawal/opt/anaconda3/envs/tf/lib/python3.10/site-packages/gymnasium/spaces/multi_discrete.py:177: UserWarning: WARN: Getting the length of a multi-dimensional MultiDiscrete space.
(RolloutWorker pid=1265)   gym.logger.warn(
2023-03-11 00:38:37,369	INFO trainable.py:172 -- Trainable.setup took 14.660 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-03-11 00:38:37

  0: Min/Mean/Max reward/episode length mean:      nan/     nan/     nan/     nan. Checkpoint saved to tmp/dqn/towerofhanoi/checkpoint_000002


2023-03-11 00:38:40,651	WARNING policy.py:126 -- Can not figure out a durable policy name for <class 'ray.rllib.policy.eager_tf_policy.SACTFPolicy_eager'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoint needs to be loaded in the future. To fix this, make sure you add your custom policy in rllib.algorithms.registry.POLICIES.


  1: Min/Mean/Max reward/episode length mean:      nan/     nan/     nan/     nan. Checkpoint saved to tmp/dqn/towerofhanoi/checkpoint_000003


2023-03-11 00:38:41,810	WARNING policy.py:126 -- Can not figure out a durable policy name for <class 'ray.rllib.policy.eager_tf_policy.SACTFPolicy_eager'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoint needs to be loaded in the future. To fix this, make sure you add your custom policy in rllib.algorithms.registry.POLICIES.


...

2023-03-11 00:39:14,260	WARNING policy.py:126 -- Can not figure out a durable policy name for <class 'ray.rllib.policy.eager_tf_policy.SACTFPolicy_eager'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoint needs to be loaded in the future. To fix this, make sure you add your custom policy in rllib.algorithms.registry.POLICIES.


  7: Min/Mean/Max reward/episode length mean:      nan/     nan/     nan/     nan. Checkpoint saved to tmp/dqn/towerofhanoi/checkpoint_000009


2023-03-11 00:39:24,340	WARNING policy.py:126 -- Can not figure out a durable policy name for <class 'ray.rllib.policy.eager_tf_policy.SACTFPolicy_eager'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoint needs to be loaded in the future. To fix this, make sure you add your custom policy in rllib.algorithms.registry.POLICIES.


  8: Min/Mean/Max reward/episode length mean:      nan/     nan/     nan/     nan. Checkpoint saved to tmp/dqn/towerofhanoi/checkpoint_000010


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data=episode_data)
df.plot(x="n", y=["episode_reward_mean", "episode_reward_min", "episode_reward_max"], secondary_y=True)

In [ ]:
agent = ppo.PPO(env=TowerOfHanoiEnv, config={
    "env_config": {"num_disks": 2},
    "num_workers": 2,
    #"train_batch_size": 16,
    "lr": 0.00005,
    "model": {
        "fcnet_hiddens": [128, 256],
        "fcnet_activation": "relu",
    },

})

results = []
episode_data = []
episode_json = []

for n in range(30):
    result = agent.train()
    results.append(result)

    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"],
        "episode_reward_max": result["episode_reward_max"],
        "episode_len_mean": result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)

    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

In [ ]:
df = pd.DataFrame(data=episode_data)
df.plot(x="n", y=["episode_reward_mean", "episode_reward_min", "episode_reward_max"], secondary_y=True)

In [ ]:
agent =DQN(env=TowerOfHanoiEnv, config={
    #"model": {
    #    "custom_preprocessor": MyPreprocessor,
        # other model configuration parameters
    #}
    # other algorithm configuration parameters
    "env_config": {"num_disks": 1},
    #"num_workers": 2,
    "train_batch_size": 16,
    "lr": 0.00005,

})

results = []
episode_data = []
episode_json = []

for n in range(30):
    result = agent.train()
    results.append(result)

    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"],
        "episode_reward_max": result["episode_reward_max"],
        "episode_len_mean": result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)

    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

In [ ]:
df = pd.DataFrame(data=episode_data)
df.plot(x="n", y=["episode_reward_mean", "episode_reward_min", "episode_reward_max"], secondary_y=True)

In [ ]:
agent =DQN(env=TowerOfHanoiEnv, config={
    "env_config": {"num_disks": 5},
    #"num_workers": 2,
    "train_batch_size": 16,
    "lr": 0.00005,

})

results = []
episode_data = []
episode_json = []

for n in range(30):
    result = agent.train()
    results.append(result)

    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"],
        "episode_reward_max": result["episode_reward_max"],
        "episode_len_mean": result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)

    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

# Custom Preprocessor - Flatten Observation Space

In [ ]:
# Register gym env in rllib

def env_creator(env_config):
    return FlattenEnv(TowerOfHanoiEnv(env_config))

register_env("TowerOfHanoiEnv", env_creator)

In [ ]:
agent =DQN(env=TowerOfHanoiEnv, config={

    "env_config": {"num_disks": 1},
    #"num_workers": 2,
    "train_batch_size": 16,
    "lr": 0.00005,

})

results = []
episode_data = []
episode_json = []

for n in range(30):
    result = agent.train()
    results.append(result)

    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"],
        "episode_reward_max": result["episode_reward_max"],
        "episode_len_mean": result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)

    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

# Custom Preprocessor - Embeddings

In [1]:
import tensorflow as tf
import numpy as np
from preprocessing import AutoEncoder, Dataset
from towerofhanoienv import TowerOfHanoiEnv


2023-03-10 18:13:05.089593: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [44]:
train_dataset = Dataset("Data/towerofHanoi_40k.npy", batch_size=8)
validation_dataset = Dataset("Data/towerofHanoi_10k.npy", batch_size=2)
#test_dataset = Dataset("Data/towerofHanoi_10k.npy", batch_size=64)

In [45]:
model = AutoEncoder()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1.2e-6),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.Precision()])



model.fit(train_dataset, epochs=50, validation_data=validation_dataset, shuffle=True)


Epoch 1/50
5000/5000 [==============================] - 14s 2ms/step - loss: 0.7304 - precision_26: 0.2408 - val_loss: 0.7319 - val_precision_26: 0.2500
Epoch 2/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.6971 - precision_26: 0.6217 - val_loss: 0.7152 - val_precision_26: 0.5000
Epoch 3/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.6761 - precision_26: 0.8992 - val_loss: 0.7064 - val_precision_26: 0.5000
Epoch 4/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.6619 - precision_26: 0.9460 - val_loss: 0.7054 - val_precision_26: 0.5455
Epoch 5/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.6537 - precision_26: 1.0000 - val_loss: 0.7102 - val_precision_26: 0.5455
Epoch 6/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.6480 - precision_26: 1.0000 - val_loss: 0.7175 - val_precision_26: 0.5455
Epoch 7/50
5000/5000 [==============================] - 12s 2ms/step - loss: 0.643

KeyboardInterrupt: 

In [46]:
import pickle
weights = model.get_weights()
with open('weights.pkl', 'wb') as f:
    pickle.dump(weights, f)


In [47]:
import gymnasium
from preprocessing import AutoEncoder, Dataset
from towerofhanoienv import TowerOfHanoiEnv
import numpy as np
import tensorflow as tf
import ray

class EmbeddingEnv(gymnasium.ObservationWrapper):
# Override `observation` to custom process the original observation
# coming from the env.
    def __init__(self, env):
         super().__init__(env)
         self.observation_space = gymnasium.spaces.Box(shape=(8,), low=-np.inf, high=np.inf)
    def observation(self, observation):
        preprocess = ray.rllib.utils.numpy.one_hot(observation)
        preprocess = preprocess.reshape(-1, 330)
        model =  tf.keras.models.load_model('saved_model/autoencoder')
        output = model.encoder(preprocess)
        print(output)
        return output
    
    
from ray.tune.registry import register_env

def env_creator(env_config):
    return EmbeddingEnv(TowerOfHanoiEnv(env_config))

register_env("TowerOfHanoiEnv", env_creator)

In [85]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig
from ray.tune.logger import pretty_print
import tensorflow as tf
import json

from towerofhanoienv import TowerOfHanoiEnv


checkpoint_root = "tmp/dqn/towerofhanoi";
algo = DQNConfig().environment(TowerOfHanoiEnv, env_config={"num_disks": 5}).training(model={"_disable_preprocessor_api": True,"use_lstm":True}, lr=0.01).build()
#algo = DQNConfig().environment(TowerOfHanoiEnv, env_config={"num_disks": 5}).rollouts(num_rollout_workers=1).training(model={"fcnet_hiddens":[256,128,64], "fcnet_activation":"tanh", "_disable_preprocessor_api": True,"use_attention":True}, lr=0.01).build()

results = []
episode_data = []
episode_json = []

for n in range(5):
    result = algo.train()
    results.append(result)

    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"],
        "episode_reward_max": result["episode_reward_max"],
        "episode_len_mean": result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = algo.save(checkpoint_root)

    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

2023-03-10 23:07:20,865	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-03-10 23:07:20,866	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.


AssertionError: 

In [80]:
results[3]

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'default_policy': {'learner_stats': {'cur_lr': 0.009999999776482582,
     'mean_q': -7.8514757,
     'min_q': -8.262798,
     'max_q': -7.2481847,
     'mean_td_error': -0.034461588,
     'model': {}},
    'td_error': array([ 0.04377365, -0.19527864, -0.20345926, -0.01094389,  0.13382912,
           -0.16710758, -0.05218697, -0.2419281 , -0.12647867,  0.11384296,
           -0.10818195,  0.13382912,  0.00939035, -0.17485619, -0.17335749,
            0.02789545, -0.18329859,  0.10252094, -0.0297308 , -0.06614017,
            0.04377365,  0.00196362,  0.02789545, -0.03977776,  0.02482843,
           -0.11675215, -0.05057478, -0.09322596,  0.0444994 ,  0.02554703,
           -0.19767904,  0.394598  ], dtype=float32),
    'custom_metrics': {},
    'num_agent_steps_trained': 32.0,
    'num_grad_updates_lifetime': 3000.0,
    'diff_num_grad_updates_vs_sampler_policy': 2999.0}},
  'num_env_steps_sampled': 4000,
  'num_env_step

In [63]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig
config = DQNConfig()
print(config.exploration_config)  
explore_config = config.exploration_config.update( 
    {
        "initial_epsilon": 1.5,
        "final_epsilon": 0.01,
        "epsilone_timesteps": 5000,
    }
)


{'type': 'EpsilonGreedy', 'initial_epsilon': 1.0, 'final_epsilon': 0.02, 'epsilon_timesteps': 10000}
